In [1]:
import numpy as np
import pandas as pd
import ipynb.fs.full.get_artists_info_api as getter

In [38]:
tracks_id = []
for i in range(len(getter.tracks)):
    tracks_id.append(getter.tracks[i]['id'])

tracks_id = list(dict.fromkeys(tracks_id))

In [111]:
track_features_list = []

for i in range(len(tracks_id)):
    track_features_list.append(getter.spotify.audio_features(tracks_id[i]))

In [112]:
#there are some tracks without their audio features so we need to remove them from the list
track_features_list = list(filter(lambda x: x != [None], track_features_list))

In [113]:
for i in range(len(track_features_list)):
    track_features_list[i] = dict(list(filter(lambda x: x[0] == 'id' or type(x[1]) != str, list(track_features_list[i][0].items()))))

In [164]:
artists = list(filter(lambda x: x['genres'] != '' ,getter.artists))

In [169]:
genres_list = []

for i in range(len(artists)):
    keys = ['id', 'genre']
    genre = dict.fromkeys(keys)
    for j in range(len(getter.tracks)):
        if artists[i]['id'] == getter.tracks[j]['artists'][0]['id']:
            genre['id'] = getter.tracks[j]['id']
            genre['genre'] = artists[i]['genres'].split(', ')
            genre['genre'] = genre['genre'][0]
            genres_list.append(genre)

In [196]:
for i in range(len(track_features_list)):
    for j in range(len(genres_list)):
        if track_features_list[i]['id'] == genres_list[j]['id']:
            track_features_list[i]['genre'] = genres_list[j]['genre']

In [201]:
track_features_list = list(filter(lambda x: len(x) == 15, track_features_list))

In [203]:
len(track_features_list)

294